### 01.Packages

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import undetected_chromedriver as uc

from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
from datetime import datetime

import requests
from bs4 import BeautifulSoup


### 02.코스메 한국 브랜드 랭킹

In [2]:
brand_names = []
product_names = []
category_names = []
rating_list = []
review_list = []
price_list = []
onsale_date = []
url_list = []

# 사이트 이동
for page_num in range(1, 6):
    # url = base_url + str(page_num)
    url = f"https://www.cosme.net/categories/pickup/1039/ranking/?page={page_num}"

    response = requests.get(url)
    time.sleep(random.uniform(5, 10))
    soup = BeautifulSoup(response.text, "html.parser")

    # '집계기간'이 포함된 태그 찾기
    date_range_tag = soup.find(string=lambda text: "集計期間" in text)

    # 찾은 태그에서 텍스트 추출
    if date_range_tag:
        date_range = date_range_tag.strip()[5:]  
    
    # 브랜드
    brands = soup.find_all("span", class_="brand")
    for span in brands:
        a_tag = span.find("a")  # <span> 안의 <a> 태그 추출
        if a_tag:
            brand_names.append(a_tag.text)
    
    # 아이템
    items = soup.find_all("h4", class_="item")
    for span in items:
        a_tag = span.find("a")  
        if a_tag:
            product_names.append(a_tag.text)

    # 카테고리
    categorys = soup.find_all("span", class_="category")
    for span in categorys:
        a_tag = span.find("a")  
        if a_tag:
            category_names.append(a_tag.text)

    # 평점
    rating_point_tags = soup.find_all("div", class_="rating-point clearfix")
    for rating_point_tag in rating_point_tags:
        rating_tag = rating_point_tag.find("p", class_=["rating", "reviewer-average", "arg-5", "arg-6"])
        if rating_tag:
            rating = rating_tag.get_text().strip()  # 평점 텍스트 추출
            rating_list.append(rating)

    # 리뷰 수
    review_tags = soup.find_all("a", class_="count")
    for review_tag in review_tags:
        review_count = review_tag.get_text().strip()  # 리뷰 수 텍스트
        review_list.append(review_count)

    # URL
    urls = soup.find_all("dd", class_="pic")

    for url in urls:
        a_tag = url.find("a")  # <a> 태그를 찾기
        if a_tag:
            url_list.append(a_tag.get("href"))  # href 속성 추출

    # 가격과 발매일 정보를 모두 추출
    price_tags = soup.find_all("p", class_="price")
    release_date_tags = soup.find_all("p", class_="onsale")

    # 가격과 발매일 정보가 각각 동일한 개수로 존재한다고 가정
    for price_tag, release_date_tag in zip(price_tags, release_date_tags):
        price = price_tag.get_text().strip()  # 가격 텍스트
        release_date = release_date_tag.get_text().strip()  # 발매일 텍스트
        price_list.append(price)
        onsale_date.append(release_date)
    
    print(f'page : {page_num} DONE')

print("-" * 50)
print(f"brand_names 길이: {len(brand_names)}")
print(f"product_names 길이: {len(product_names)}")
print(f"category_names 길이: {len(category_names)}")
print(f"rating_list 길이: {len(rating_list)}")
print(f"review_list 길이: {len(review_list)}")
print(f"price_list 길이: {len(price_list)}")
print(f"onsale_date 길이: {len(onsale_date)}")
print(f"url_list 길이: {len(url_list)}")

page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
--------------------------------------------------
brand_names 길이: 50
product_names 길이: 50
category_names 길이: 50
rating_list 길이: 50
review_list 길이: 50
price_list 길이: 50
onsale_date 길이: 50
url_list 길이: 50


In [3]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

df = pd.DataFrame({
    "브랜드" : brand_names,
    "제품명" : product_names,
    "카테고리" : category_names,
    "평점" : rating_list,
    "리뷰수" : review_list,
    "가격(円)" : price_list,
    "발매일" : onsale_date,
    'url' : url_list,
    '업로드일' : now
})
# 순위
df = df.reset_index()
df.rename(columns = {'index' : '순위'}, inplace=True)
df['순위'] = df['순위'] + 1

# 발매일에서 날짜 부분만 추출
df['발매일'] = df['발매일'].apply(lambda x: re.sub(r'発売日：', '', x))

# URL에서 상품 ID 추출
df['상품ID'] = df['url'].apply(lambda x: re.search(r'products/(\d+)', x).group(1))

# 가격 전처리 함수
def extract_price(price_text):
    # 정규표현식을 사용하여 숫자만 추출
    price_numbers = re.findall(r'(\d+,?\d*)', price_text)
    if price_numbers:
        # 쉼표가 있는 경우 처리
        final_price = price_numbers[-1].replace(',', '')
        return final_price
    return "0"  # 숫자가 없는 경우 기본값

# 가격 컬럼 전처리
df['가격(円)'] = df['가격(円)'].apply(extract_price)
print(df.shape)
df.head()

(50, 11)


,순위,브랜드,제품명,카테고리,평점,리뷰수,가격(円),발매일,url,업로드일,상품ID
0,1,ナンバーズイン(numbuzin),5番 白玉グルタチオンCふりかけマスク,シートマスク・パック,5.2,クチコミ6484件,1090,2024/1/15,https://www.cosme.net/products/10245780/,2025-02-21 00:43:00,10245780
1,2,rom&nd,ハンオールブロウカラ,眉マスカラ,5.4,クチコミ5885件,1210,2023/7/27 (2024/3/1追加発売),https://www.cosme.net/products/10235626/,2025-02-21 00:43:00,10235626
2,3,Torriden (トリデン),ダイブイン マスク,シートマスク・パック,5.4,クチコミ4721件,825,- (2023/6/20追加発売),https://www.cosme.net/products/10210539/,2025-02-21 00:43:00,10210539
3,4,fwee(フィー),フィー 3Dボリューミンググロス,リップグロス,5.4,クチコミ1580件,1760,2024/7/1,https://www.cosme.net/products/10254107/,2025-02-21 00:43:00,10254107
4,5,ナンバーズイン(numbuzin),3番 すべすべキメケアシートマスク,シートマスク・パック,5.0,クチコミ4396件,1090,2021/9/10 (2022/11/28追加発売),https://www.cosme.net/products/10216004/,2025-02-21 00:43:00,10216004


In [9]:
# 날짜 범위를 가져온 후, 슬래시(/)를 대시(-)로 변환
safe_date_range = date_range.replace("/", "-").replace("〜", "_")

df.to_csv(f"cosme_korea/{safe_date_range}_korea_cosme_ranking.csv", index=False, encoding="utf-8-sig")

### 03.코스메 한국 브랜드 제품 상세

In [10]:
# 제품 정보를 저장할 리스트 생성
all_product_details = []

# 각 제품에 대해 정보 수집
for product_id in df['상품ID'].to_list():  
    url = f"https://www.cosme.net/products/{product_id}/"

    # 정보 저장할 딕셔너리 생성
    product_details = {'product_id': product_id}  # product_id는 첫 번째 열로 추가

    # 요청 보내고 응답 받기
    response = requests.get(url)
    time.sleep(random.uniform(5, 10))  # 요청 간격 두기
    soup = BeautifulSoup(response.text, "html.parser")

    # 특정 ID와 클래스명을 가진 div 태그 찾기
    maker = soup.find("dl", class_="maker clearfix")

    # maker 정보 추출
    if maker:
        maker_name = maker.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['メーカー'] = maker_name  # "メーカー" 항목에 해당하는 값 저장

    # 브랜드 정보 추출
    brand = soup.find("dl", class_="brand-name clearfix")

    if brand:
        brand_name = brand.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['ブランド名'] = brand_name

    # 아이템 카테고리 정보 추출
    category = soup.find("dl", class_="item-category clearfix")

    if category:
        # 모든 <a> 태그의 텍스트를 가져와서 " > "로 연결
        category_names = " > ".join([a.text.strip() for a in category.find_all("a")])
        product_details['アイテムカテゴリ'] = category_names

    # 베스트 코スメ 어워드 정보 추출
    bestcosme = soup.find("dl", class_="bestcosme clearfix")

    if bestcosme:
        # <ul> 내 모든 <li> 항목을 가져오기
        bestcosme_awards = {}
        for idx, li in enumerate(bestcosme.find_all("li"), start=1):
            award_text = li.find("a").text.strip()  # 링크 텍스트
            award_url = li.find("a")["href"]  # 링크 URL
            bestcosme_awards[f"BestCosme_Award_{idx}"] = award_text

        # BestCosme 항목 추가
        product_details['ベストコスメ'] = bestcosme_awards

    # 랭킹IN 정보 추출
    rankingIn = soup.find("dl", class_="ranking-in clearfix")

    if rankingIn:
        product_details['ランキングIN'] = rankingIn.find("a").text.strip().replace("\u3000", " ")

    # 제품 정보 리스트에 추가
    all_product_details.append(product_details)

In [11]:
# all_product_details 리스트를 DataFrame으로 변환
df_detail = pd.DataFrame(all_product_details)
df_detail.head()

,product_id,メーカー,ブランド名,アイテムカテゴリ,ベストコスメ,ランキングIN
0,10245780,BENOW,ナンバーズイン(numbuzin),スキンケア・基礎化粧品 > パック・フェイスマスク > シートマスク・パック,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 ベス...,シートマスク・パックランキング 1位
1,10235626,iFAMILYSC,rom&nd,メイクアップ > アイブロウ > 眉マスカラ,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 総合...,眉マスカラランキング 1位
2,10210539,Torriden,Torriden (トリデン),スキンケア・基礎化粧品 > パック・フェイスマスク > シートマスク・パック,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 ベス...,シートマスク・パックランキング 3位
3,10254107,BENOW,fwee(フィー),メイクアップ > 口紅・グロス・リップライナー > リップグロス,{'BestCosme_Award_1': '@cosmeベストコスメアワード2024 下半...,リップグロスランキング 1位
4,10216004,BENOW,ナンバーズイン(numbuzin),スキンケア・基礎化粧品 > パック・フェイスマスク > シートマスク・パック,NaN,韓国コスメランキング 5位


In [12]:
# DataFrame을 CSV로 저장
df_detail.to_csv("cosme_korea/korea_cosme_ranking_details.csv", index=False, encoding="utf-8-sig")

### 04.코스메 한국 브랜드 제품 (리뷰수 내림차순)

In [13]:
from tqdm import tqdm

In [14]:
brand_names = []
product_names = []
category_names = []
rating_list = []
review_list = []
price_list = []
onsale_date = []
url_list = []

# 사이트 이동 (tqdm 적용)
for page_num in tqdm(range(1, 301), desc="Scraping Progress", unit="page"):
    url = f"https://www.cosme.net/categories/pickup/1039/product/?page={page_num}&sort=review"
    
    response = requests.get(url)
    time.sleep(random.uniform(5, 10))
    soup = BeautifulSoup(response.text, "html.parser")

    # 브랜드
    brands = soup.find_all("span", class_="brand")
    for span in brands:
        a_tag = span.find("a")
        if a_tag:
            brand_names.append(a_tag.text)
    
    # 아이템
    items = soup.find_all("h3", class_="item")
    for span in items:
        a_tag = span.find("a")  
        if a_tag:
            product_names.append(a_tag.text)

    # 카테고리
    categorys = soup.find_all("p", class_="category")
    for span in categorys:
        a_tag = span.find("a")  
        if a_tag:
            category_names.append(a_tag.text)

    # 평점
    rating_point_tags = soup.find_all('span', class_=lambda x: x and 'value reviewer-average' in x)
    for rating_point_tag in rating_point_tags:
        if rating_point_tag:
            rating = rating_point_tag.get_text().strip()
            rating_list.append(rating)

    # 리뷰 수
    review_tags = soup.find_all("a", class_="count")
    for review_tag in review_tags:
        review_count = review_tag.get_text().strip()
        review_list.append(review_count)

    # URL
    urls = soup.find_all("p", class_="pic")
    for url in urls:
        a_tag = url.find("a")
        if a_tag:
            url_list.append(a_tag.get("href"))

    # 가격과 발매일 정보 추출
    price_tags = soup.find_all("span", class_="price")
    release_date_tags = soup.find_all("span", class_="sell")
    
    for price_tag, release_date_tag in zip(price_tags, release_date_tags):
        price = price_tag.get_text().strip()
        release_date = release_date_tag.get_text().strip()
        price_list.append(price)
        onsale_date.append(release_date)

Scraping Progress: 100%|██████████| 300/300 [52:30<00:00, 10.50s/page]


In [15]:
now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

df = pd.DataFrame({
    "브랜드" : brand_names,
    "제품명" : product_names,
    "카테고리" : category_names,
    "평점" : rating_list,
    "리뷰수" : review_list,
    "가격(円)" : price_list,
    "발매일" : onsale_date,
    'url' : url_list,
    '업로드일' : now
})
# 순위
df = df.reset_index()
df.rename(columns = {'index' : '순위'}, inplace=True)
df['순위'] = df['순위'] + 1

# 발매일에서 날짜 부분만 추출
df['발매일'] = df['발매일'].apply(lambda x: re.sub(r'発売日：', '', x))

# URL에서 상품 ID 추출
df['상품ID'] = df['url'].apply(lambda x: re.search(r'products/(\d+)', x).group(1))

# 가격 전처리 함수
def extract_price(price_text):
    # 정규표현식을 사용하여 숫자만 추출
    price_numbers = re.findall(r'(\d+,?\d*)', price_text)
    if price_numbers:
        # 쉼표가 있는 경우 처리
        final_price = price_numbers[-1].replace(',', '')
        return final_price
    return "0"  # 숫자가 없는 경우 기본값

# 가격 컬럼 전처리
df['가격(円)'] = df['가격(円)'].apply(extract_price)
print(df.shape)
df.head()

(3000, 11)


,순위,브랜드,제품명,카테고리,평점,리뷰수,가격(円),발매일,url,업로드일,상품ID
0,1,VT(ブイティー),CICA デイリースージングマスク,シートマスク・パック,5.2,クチコミ10040件,2420,2019/12/2,https://www.cosme.net/products/10189359/,2025-02-21 01:44:59,10189359
1,2,the SAEM,CPチップコンシーラー,コンシーラー,4.6,クチコミ8808件,858,- (2018/8/22追加発売),https://www.cosme.net/products/10145207/,2025-02-21 01:44:59,10145207
2,3,rom&nd,ジューシーラスティングティント,口紅,4.9,クチコミ6715件,1320,2019/11/21 (2024/4/20追加発売),https://www.cosme.net/products/10176465/,2025-02-21 01:44:59,10176465
3,4,TIRTIR,MASK FIT RED CUSHION,リキッドファンデーション,4.8,クチコミ6539件,2970,2021/12/11 (2024/7/17追加発売),https://www.cosme.net/products/10217471/,2025-02-21 01:44:59,10217471
4,5,ナンバーズイン(numbuzin),5番 白玉グルタチオンCふりかけマスク,シートマスク・パック,5.2,クチコミ6484件,1090,2024/1/15,https://www.cosme.net/products/10245780/,2025-02-21 01:44:59,10245780


In [16]:
df.to_csv("cosme_korea/korea_cosme_product_3000.csv", index=False, encoding="utf-8")

### 05.코스메 한국 브랜드 제품 (리뷰수 내림차순) 상세

In [26]:
df = pd.read_csv("cosme_korea/korea_cosme_product_3000.csv")

In [27]:
# 제품 정보를 저장할 리스트 생성
all_product_details = []

# 각 제품에 대해 정보 수집 (tqdm 적용)
for product_id in tqdm(df['상품ID'].to_list(), desc="Product Scraping", unit="product"):
    url = f"https://www.cosme.net/products/{product_id}/"
    
    product_details = {'product_id': product_id}  
    response = requests.get(url)
    time.sleep(random.uniform(5, 10)) # 3~5 해도 될 듯
    soup = BeautifulSoup(response.text, "html.parser")

    maker = soup.find("dl", class_="maker clearfix")
    if maker:
        maker_name = maker.find("a").text.strip()
        product_details['メーカー'] = maker_name

    brand = soup.find("dl", class_="brand-name clearfix")
    if brand:
        brand_name = brand.find("a").text.strip()
        product_details['ブランド名'] = brand_name

    category = soup.find("dl", class_="item-category clearfix")
    if category:
        category_names = " > ".join([a.text.strip() for a in category.find_all("a")])
        product_details['アイテムカテゴリ'] = category_names

    bestcosme = soup.find("dl", class_="bestcosme clearfix")
    if bestcosme:
        bestcosme_awards = {}
        for idx, li in enumerate(bestcosme.find_all("li"), start=1):
            award_text = li.find("a").text.strip()
            award_url = li.find("a")["href"]
            bestcosme_awards[f"BestCosme_Award_{idx}"] = award_text
        product_details['ベストコスメ'] = bestcosme_awards

    rankingIn = soup.find("dl", class_="ranking-in clearfix")
    if rankingIn:
        product_details['ランキングIN'] = rankingIn.find("a").text.strip().replace("\u3000", " ")

    all_product_details.append(product_details)

# DataFrame 변환 및 저장
df_detail = pd.DataFrame(all_product_details)
df_detail.to_csv("cosme_korea/korea_cosme_product_3000_details.csv", index=False, encoding="utf-8")

Product Scraping: 100%|██████████| 3000/3000 [8:12:12<00:00,  9.84s/product]  


### 06.코스메 일본 카테고리별 랭킹

In [21]:
categories = {
    "스킨케어 전체 순위": 900,
    "클렌징": 901,
    "화장수" : 902,
    "미용액_유액" : 903,
    "팩_페이스마스크": 904,
    "눈가_입원_케어": 905,
    "기타_스킨케어": 906,
    "썬크림": 801,
    "메이크업": 802,
    "베이스_메이크업": 803
}

In [22]:
brand_names = []
product_names = []
category_names = []
rating_list = []
review_list = []
price_list = []
onsale_date = []
url_list = []

# 카테고리별로 순위 출력
for item, code in categories.items():
    # 사이트 이동
    for page_num in range(1, 6):
        url = f"https://www.cosme.net/categories/item/{code}/ranking/?page={page_num}"

        response = requests.get(url)
        time.sleep(random.uniform(5, 10))
        soup = BeautifulSoup(response.text, "html.parser")

        # '집계기간'이 포함된 태그 찾기
        date_range_tag = soup.find(string=lambda text: "集計期間" in text)

        # 찾은 태그에서 텍스트 추출
        if date_range_tag:
            date_range = date_range_tag.strip()[5:]  
        
        # 브랜드
        brands = soup.find_all("span", class_="brand")
        for span in brands:
            a_tag = span.find("a")  # <span> 안의 <a> 태그 추출
            if a_tag:
                brand_names.append(a_tag.text)
        
        # 아이템
        items = soup.find_all("h4", class_="item")
        for span in items:
            a_tag = span.find("a")  
            if a_tag:
                product_names.append(a_tag.text)

        # 카테고리
        categorys = soup.find_all("span", class_="category")
        for span in categorys:
            a_tag = span.find("a")  
            if a_tag:
                category_names.append(a_tag.text)

        # 평점
        rating_point_tags = soup.find_all('span', class_=lambda x: x and 'reviewer-average' in x)
        for rating_point_tag in rating_point_tags:
            if rating_point_tag:  # 올바른 변수명 사용
                rating = rating_point_tag.get_text().strip()  # 평점 텍스트 추출
                rating_list.append(rating)

        # 리뷰 수
        review_tags = soup.find_all("a", class_="count")
        for review_tag in review_tags:
            review_count = review_tag.get_text().strip()  # 리뷰 수 텍스트
            review_list.append(review_count)

        # URL
        urls = soup.find_all("dd", class_="pic")

        for url in urls:
            a_tag = url.find("a")  # <a> 태그를 찾기
            if a_tag:
                url_list.append(a_tag.get("href"))  # href 속성 추출

        # 가격과 발매일 정보를 모두 추출
        price_tags = soup.find_all("p", class_="price")
        release_date_tags = soup.find_all("p", class_="onsale")

        # 가격과 발매일 정보가 각각 동일한 개수로 존재한다고 가정
        for price_tag, release_date_tag in zip(price_tags, release_date_tags):
            price = price_tag.get_text().strip()  # 가격 텍스트
            release_date = release_date_tag.get_text().strip()  # 발매일 텍스트
            price_list.append(price)
            onsale_date.append(release_date)
        
        print(f'page : {page_num} DONE')

    print(f"brand_names 길이: {len(brand_names)}")
    print(f"product_names 길이: {len(product_names)}")
    print(f"category_names 길이: {len(category_names)}")
    print(f"rating_list 길이: {len(rating_list)}")
    print(f"review_list 길이: {len(review_list)}")
    print(f"price_list 길이: {len(price_list)}")
    print(f"onsale_date 길이: {len(onsale_date)}")
    print(f"url_list 길이: {len(url_list)}")

    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    df = pd.DataFrame({
        "브랜드" : brand_names,
        "제품명" : product_names,
        "카테고리" : category_names,
        "평점" : rating_list,
        "리뷰수" : review_list,
        "가격(円)" : price_list,
        "발매일" : onsale_date,
        'url' : url_list,
        '업로드일' : now
    })
    # 순위
    df = df.reset_index()
    df.rename(columns = {'index' : '순위'}, inplace=True)
    df['순위'] = df['순위'] + 1

    # 발매일에서 날짜 부분만 추출
    df['발매일'] = df['발매일'].apply(lambda x: re.sub(r'発売日：', '', x))

    # URL에서 상품 ID 추출
    df['상품ID'] = df['url'].apply(lambda x: re.search(r'products/(\d+)', x).group(1))

    # 가격 컬럼 전처리
    df['가격(円)'] = df['가격(円)'].apply(extract_price)
    print(df.shape)
    df.to_csv(f"cosme_category/{item}_cosme_ranking.csv", index=False, encoding="utf-8-sig")

page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
brand_names 길이: 50
product_names 길이: 50
category_names 길이: 50
rating_list 길이: 50
review_list 길이: 50
price_list 길이: 50
onsale_date 길이: 50
url_list 길이: 50
(50, 11)
page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
brand_names 길이: 100
product_names 길이: 100
category_names 길이: 100
rating_list 길이: 100
review_list 길이: 100
price_list 길이: 100
onsale_date 길이: 100
url_list 길이: 100
(100, 11)
page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
brand_names 길이: 150
product_names 길이: 150
category_names 길이: 150
rating_list 길이: 150
review_list 길이: 150
price_list 길이: 150
onsale_date 길이: 150
url_list 길이: 150
(150, 11)
page : 1 DONE
page : 2 DONE
page : 3 DONE
page : 4 DONE
page : 5 DONE
brand_names 길이: 200
product_names 길이: 200
category_names 길이: 200
rating_list 길이: 200
review_list 길이: 200
price_list 길이: 200
onsale_date 길이: 200
url_list 길이: 200
(200, 11)
page : 1 DONE
page : 2 DONE
page : 3 DONE
pag

In [23]:
import glob

# 파일 경로 패턴 설정
file_pattern = "cosme_category/*_cosme_ranking.csv"

# 모든 CSV 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

# 상품 ID 저장할 리스트
product_ids = []

# 각 파일 읽어서 상품 ID 추출
for file in file_list:
    df = pd.read_csv(file)
    
    # '상품 ID' 컬럼이 있다고 가정하고 추출
    if '상품ID' in df.columns:
        product_ids.extend(df['상품ID'].dropna().astype(str).tolist())

# 중복 제거 후 리스트 변환
product_ids = list(set(product_ids))
print("총 상품 개수:", len(product_ids))

총 상품 개수: 455


In [24]:
# 제품 정보를 저장할 리스트 생성
all_product_details = []

# 각 제품에 대해 정보 수집
for product_id in product_ids:  
    url = f"https://www.cosme.net/products/{product_id}/"

    # 정보 저장할 딕셔너리 생성
    product_details = {'product_id': product_id}  # product_id는 첫 번째 열로 추가

    # 요청 보내고 응답 받기
    response = requests.get(url)
    time.sleep(random.uniform(5, 10))  # 요청 간격 두기
    soup = BeautifulSoup(response.text, "html.parser")

    # 특정 ID와 클래스명을 가진 div 태그 찾기
    maker = soup.find("dl", class_="maker clearfix")

    # maker 정보 추출
    if maker:
        maker_name = maker.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['メーカー'] = maker_name  # "メーカー" 항목에 해당하는 값 저장

    # 브랜드 정보 추출
    brand = soup.find("dl", class_="brand-name clearfix")

    if brand:
        brand_name = brand.find("a").text.strip()  # <a> 태그의 텍스트 내용
        product_details['ブランド名'] = brand_name

    # 아이템 카테고리 정보 추출
    category = soup.find("dl", class_="item-category clearfix")

    if category:
        # 모든 <a> 태그의 텍스트를 가져와서 " > "로 연결
        category_names = " > ".join([a.text.strip() for a in category.find_all("a")])
        product_details['アイテムカテゴリ'] = category_names

    # 베스트 코スメ 어워드 정보 추출
    bestcosme = soup.find("dl", class_="bestcosme clearfix")

    if bestcosme:
        # <ul> 내 모든 <li> 항목을 가져오기
        bestcosme_awards = {}
        for idx, li in enumerate(bestcosme.find_all("li"), start=1):
            award_text = li.find("a").text.strip()  # 링크 텍스트
            award_url = li.find("a")["href"]  # 링크 URL
            bestcosme_awards[f"BestCosme_Award_{idx}"] = award_text

        # BestCosme 항목 추가
        product_details['ベストコスメ'] = bestcosme_awards

    # 랭킹IN 정보 추출
    rankingIn = soup.find("dl", class_="ranking-in clearfix")

    if rankingIn:
        product_details['ランキングIN'] = rankingIn.find("a").text.strip().replace("\u3000", " ")

    # 제품 정보 리스트에 추가
    all_product_details.append(product_details)

In [25]:
# all_product_details 리스트를 DataFrame으로 변환
df_detail = pd.DataFrame(all_product_details)
df_detail.head()
# DataFrame을 CSV로 저장
df_detail.to_csv("cosme_category/cosme_category_product_details.csv", index=False, encoding="utf-8-sig")